<a href="https://colab.research.google.com/github/usamabaig1/Portfolio_Optimization/blob/main/PortfolioOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd

np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False

In [31]:
# !pip install pyportfolioopt
import pypfopt 
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [79]:
df  = pd.read_excel('Data.xlsx', sheet_name= 'Portfolios')
covM  = pd.read_excel('Data.xlsx', sheet_name= 'Covariance Matrix', index_col=0)
X = 10
print(df.head())
print(df.shape)

print(covM.head())
print(covM.shape)

   Port      Mean        Std
0     0  3.057942  12.916029
1     1  5.081918  19.262509
2     2  3.623377  13.632651
3     3  1.041958   4.181238
4     4  3.277722  15.334525
(199, 3)
          0           1           2    ...       196        197       198
0  166.990639  -14.132751  -10.662156  ... -1.227519  13.636229 -1.072147
1  -14.132751  371.415283   -5.420117  ... -9.849390  -6.912194 -3.280552
2  -10.662156   -5.420117  186.035013  ... -2.876623  -4.986987 -5.183545
3   -2.613434   -0.933441   -2.347182  ... -1.906273  -0.428490  7.791790
4   26.132892    1.640227  -11.727299  ... -6.253662  -1.466145  5.632469

[5 rows x 199 columns]
(199, 199)


In [ ]:
def minVolatility(df, covM, X):
  # get weights maximizing the Sharpe ratio
  ef = EfficientFrontier(df.Mean, covM)
  weights = ef.min_volatility()
  cleaned_weights = ef.clean_weights()
  df['volatility_Weight'] = 0

  for x in range(len(cleaned_weights)):
    df.loc[x,'volatility_Weight'] = cleaned_weights[x]

  df.sort_values(by='volatility_Weight', inplace=True, ascending=False)
  df.reset_index(inplace=True, drop=True)

  df['Weight'] = 0
  ports = []
  for x in range(len(df)):
    if x < X:
      df.loc[x,'Weight'] = 1
      ports += [df.loc[x,'Port']]
    else:
      break
  print("Selected portfolios:", ports)     
  df.sort_values(by='Port', inplace=True, ascending=True)
  df.reset_index(inplace=True, drop=True)
  df.drop(columns='volatility_Weight')
  df.head(X+2)
  df.to_csv("minVolatility.csv")



In [ ]:
def maxSharpRatio(df, covM, X):
  # get weights maximizing the Sharpe ratio
  ef = EfficientFrontier(df.Mean, covM)
  weights = ef.max_sharpe()
  cleaned_weights = ef.clean_weights()
  df['sharpe_Weight'] = 0

  for x in range(len(cleaned_weights)):
    df.loc[x,'sharpe_Weight'] = cleaned_weights[x]

  df.sort_values(by='sharpe_Weight', inplace=True, ascending=False)
  df.reset_index(inplace=True, drop=True)

  df['Weight'] = 0
  ports = []
  for x in range(len(df)):
    if x < X:
      df.loc[x,'Weight'] = 1
      ports += [df.loc[x,'Port']]
    else:
      break
  print("Selected portfolios:", ports)     
  df.sort_values(by='Port', inplace=True, ascending=True)
  df.reset_index(inplace=True, drop=True)
  df.drop(columns='sharpe_Weight')
  df.head(X+2)
  df.to_csv("maxSharpRatio.csv")



In [80]:
# get weights maximizing the Sharpe ratio
# cov_mat = risk_models.sample_cov(covM)
ef = EfficientFrontier(df.Mean, covM)
weights = ef.min_volatility()
cleaned_weights = ef.clean_weights()
cleaned_weights

OrderedDict([(0, 0.00512),
             (1, 0.00495),
             (2, 0.00495),
             (3, 0.00481),
             (4, 0.00499),
             (5, 0.00486),
             (6, 0.00498),
             (7, 0.00507),
             (8, 0.00499),
             (9, 0.00505),
             (10, 0.00501),
             (11, 0.00498),
             (12, 0.005),
             (13, 0.00509),
             (14, 0.00501),
             (15, 0.00495),
             (16, 0.00515),
             (17, 0.00529),
             (18, 0.00499),
             (19, 0.00454),
             (20, 0.00517),
             (21, 0.00499),
             (22, 0.00544),
             (23, 0.00494),
             (24, 0.00502),
             (25, 0.00577),
             (26, 0.00497),
             (27, 0.00584),
             (28, 0.00498),
             (29, 0.00501),
             (30, 0.00481),
             (31, 0.00521),
             (32, 0.00524),
             (33, 0.00502),
             (34, 0.00486),
             (35, 0.00436),
    

In [45]:
# cov_mat = risk_models.sample_cov(df_stocks)
# cov_mat

,BSX,AES,BRK-B,SEE,QQQ,SPY
BSX,0.083179,0.040808,0.032342,0.037882,0.038989,0.035822
AES,0.040808,0.107892,0.033949,0.051812,0.032877,0.034956
BRK-B,0.032342,0.033949,0.042922,0.030727,0.032213,0.032550
SEE,0.037882,0.051812,0.030727,0.099930,0.027665,0.029498
QQQ,0.038989,0.032877,0.032213,0.027665,0.044265,0.036123
SPY,0.035822,0.034956,0.032550,0.029498,0.036123,0.033583


In [10]:
## NUMBER OF ASSETS
n_assets = len(df)
print ('Total Number of assets: ', n_assets)
## NUMBER OF OBSERVATIONS
n_obs = 1000

Total Number of assets:  199


In [46]:
n = len(df)
# returns = np.asmatrix(returns)

N = 100
mus = [10**(5.0 * t/N - 1.0) for t in range(N)]

# Convert to cvxopt matrices
S = opt.matrix(covM)
pbar = opt.matrix(df.Mean)

# Create constraint matrices
G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
h = opt.matrix(0.0, (n ,1))
A = opt.matrix(1.0, (1, n))
b = opt.matrix(1.0)

# Calculate efficient frontier weights using quadratic programming
portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
              for mu in mus]
## CALCULATE RISKS AND RETURNS FOR FRONTIER
returns = [blas.dot(pbar, x) for x in portfolios]
risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
m1 = np.polyfit(returns, risks, 2)
x1 = np.sqrt(m1[2] / m1[0])
# CALCULATE THE OPTIMAL PORTFOLIO
wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
return np.asarray(wt), returns, risks

TypeError: ignored

In [47]:
covM

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198
0,0,166.990639,-14.132751,-10.662156,-2.613434,26.132892,113.038407,5.506155,-5.997001,-12.197504,17.933521,-4.727133,-4.851205,-4.127079,34.616958,-7.526261,-6.308026,-15.423758,-8.466392,-12.132742,88.365961,36.603998,-4.057306,-9.781355,15.771312,-26.781770,11.045069,-5.957783,20.008732,-8.038763,-7.292133,2.540969,55.129204,-5.296341,-33.864186,-16.552386,-6.121803,3.904144,8.881546,-4.388289,...,-3.512212,-8.792175,-10.373467,-6.039436,-1.082934,-5.708062,1.237384,6.832923,-3.157920,-6.363923,-3.418069,-3.260910,7.892077,-3.214981,-13.724044,10.394642,-12.645808,-8.089572,-10.658502,-0.990706,-3.490384,22.853736,-1.517023,-2.486867,-5.907944,-11.309868,22.704595,-28.169859,-14.113174,2.215561,-6.784833,20.283601,-10.588868,-0.013055,-6.470590,-20.342358,-0.562513,-1.227519,13.636229,-1.072147
1,1,-14.132751,371.415283,-5.420117,-0.933441,1.640227,-27.698253,-44.840057,24.816826,-9.842782,-3.134263,39.525881,-47.273634,-22.505732,-23.158301,-3.641369,-8.798519,-19.299934,10.395860,81.078847,-16.656779,-8.814348,-5.182191,-14.035708,35.407234,113.308980,-9.230006,93.141720,-11.878585,-17.310803,-8.543119,2.854611,-11.541539,-3.937275,10.466668,-13.973097,1.044244,-7.303969,-37.038917,-7.507236,...,-7.486851,-17.576420,-8.531177,-3.967788,-6.080527,-19.619777,-5.561251,-6.438385,-29.167094,-6.103615,-4.950994,-5.233356,-5.285615,-4.465697,-17.717648,-7.636230,8.056375,-13.855602,-19.519435,-5.730895,-5.088749,-3.444890,-3.485998,-10.472881,-7.846266,-23.143055,-12.082400,55.261509,-23.298246,-6.951310,-9.877971,-5.717357,28.485945,-8.864560,-3.205731,5.039770,-6.096347,-9.849390,-6.912194,-3.280552
2,2,-10.662156,-5.420117,186.035013,-2.347182,-11.727299,-17.036143,7.249494,1.027506,12.007779,-6.750390,-10.993636,40.697831,6.914013,-15.827727,-6.306805,25.181169,-15.267571,17.050545,-15.940675,-11.732247,-6.562987,-3.195013,-8.999195,-13.089026,-34.852494,-6.695948,3.591091,-7.112260,11.279961,-3.215636,-8.495299,-9.425325,-2.148286,-32.741792,-0.485494,15.490221,-1.522935,-1.570052,-0.502623,...,-4.150623,-6.006364,58.136532,0.577831,-4.607429,-8.407597,-4.478494,-5.968000,-8.524519,2.801000,5.284948,-0.689584,-4.137000,-4.024623,-9.375714,-6.025675,-16.631247,-7.235779,-14.710234,-3.700909,-9.615506,-6.671286,-1.154351,-7.210364,23.146636,-5.437857,-9.201870,-32.691416,110.345130,-4.967649,9.477416,-6.340909,-15.708857,-7.853143,-7.696662,-23.972675,-3.955662,-2.876623,-4.986987,-5.183545
3,3,-2.613434,-0.933441,-2.347182,17.500238,-2.234664,-5.601154,-9.018371,1.280448,32.061014,-1.888622,-3.671476,-9.559252,-4.778643,-1.918203,-2.328189,-1.116364,8.120692,6.023441,-1.854469,-2.647545,-1.674105,-1.516049,2.118881,-2.902636,-5.078937,-0.881881,-1.600119,-1.822056,-3.558070,25.396524,-1.522126,-2.097301,23.326615,12.038245,6.294755,0.891832,-1.926448,-7.604811,14.024343,...,-1.586119,-2.436678,-1.054993,-1.873867,2.288462,1.764748,-1.219860,-1.492538,-1.934804,-1.982462,-0.914119,-1.117280,-1.131462,-0.231503,-1.787308,-1.652776,-2.806930,-2.729552,1.090119,-0.581580,-0.178140,-1.874846,-0.497706,0.418476,-1.552063,-1.188077,0.441259,19.760895,-5.062126,-1.208524,2.871259,-1.171427,-3.510077,1.386615,6.242503,13.873224,3.042042,-1.906273,-0.428490,7.791790
4,4,26.132892,1.640227,-11.727299,-2.234664,235.382793,7.017363,-21.609120,-6.228419,-11.802384,71.384500,82.167329,-26.052809,-0.414068,-11.863580,-3.314250,-6.722883,-15.845703,-10.264084,46.034994,3.293675,-0.858837,0.605914,-9.707596,137.944597,25.806274,-2.722118,-4.214168,-6.541180,-8.896796,-7.437671,24.718024,15.089391,-5.296736,-18.142856,-15.044572,-7.112539,-8.317724,-14.532036,-4.962970,...,2.609261,-9.711252,-11.122478,-6.409501,0.027626,-12.574952,6.909735,-4.168231,-18.265513,-6.

In [35]:
return_vec = np.random.randn(5, n_obs)
# random_portfolio(return_vec) 
C = np.asmatrix(np.cov(return_vec))

In [36]:
C.shape

(5, 5)

In [20]:
return_vec

array([[-0.98544704,  1.27925143, -0.0315769 , ...,  0.07699362,
        -0.51284792, -0.05335412],
       [-0.81558632,  1.62708379, -0.66171895, ...,  0.13894957,
         1.00734989, -2.39607709],
       [-1.43059049, -1.05491705,  0.85935282, ...,  0.05547069,
         0.64402765, -1.68633917],
       [-0.83919004, -0.13228456,  0.14505534, ..., -0.45047687,
        -0.46756241, -0.61906466],
       [-0.73940369,  0.2499576 , -1.80430886, ...,  1.03733765,
        -0.53873985, -0.35921403]])